In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model

In [11]:
df_windows = pd.read_csv('final_window_labels.csv') 

In [ ]:
# Define concept columns
discrete_columns = ['periodicity', 'temporal_stability', 'coordination', 'static_posture']
continuous_columns = ['motion_intensity', 'vertical_dominance']

# Split features
X_discrete = df_windows[discrete_columns].values
X_cont = df_windows[continuous_columns].values

# One-hot encode discrete features
enc = OneHotEncoder(sparse_output=False)
X_discrete_oh = enc.fit_transform(X_discrete)

# Ensure 2D (in case of single column)
if X_discrete_oh.ndim == 1:
    X_discrete_oh = X_discrete_oh.reshape(-1, 1)

# Scale continuous features
scaler = StandardScaler()
X_cont_scaled = scaler.fit_transform(X_cont)

# Combine all features
X_final = np.hstack([X_discrete_oh, X_cont_scaled])

# Prepare labels
y_label_str = df_windows['activity'].values
activity_mapping = {act: i for i, act in enumerate(np.unique(y_label_str))}
y_label = np.array([activity_mapping[act] for act in y_label_str])

# Convert labels to categorical
num_classes = len(activity_mapping)
y_cat = tf.keras.utils.to_categorical(y_label, num_classes=num_classes)

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(
    X_final, y_cat, test_size=0.25, random_state=42, stratify=y_label
)

# Build shallow neural network
model = Sequential([
    Dense(16, activation='relu', input_shape=(X_final.shape[1],)),
    Dense(16, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=16,
    verbose=1
)

# Evaluate
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

print("Accuracy:", accuracy_score(y_test_labels, y_pred))
print("\nClassification Report:\n", classification_report(y_test_labels, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test_labels, y_pred))

c:\Users\tudor\miniconda3\envs\mod9\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 16)             │           208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 6)              │           102 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 582 (2.27 KB)

 Trainable params: 582 (2.27 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.1518 - loss: 1.8793 - val_accuracy: 0.0789 - val_loss: 1.8280
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1696 - loss: 1.8269 - val_accuracy: 0.1316 - val_loss: 1.7859
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2143 - loss: 1.7801 - val_accuracy: 0.1842 - val_loss: 1.7473
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2232 - loss: 1.7388 - val_accuracy: 0.1842 - val_loss: 1.7111
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2411 - loss: 1.7022 - val_accuracy: 0.1842 - val_loss: 1.6783
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2679 - loss: 1.6656 - val_accuracy: 0.2632 - val_loss: 1.6482
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3393 - loss: 1.6354 - val_accuracy: 0.3421 - val_loss: 1.6190
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4107 - loss: 1.6031 - val_accuracy: 0.3421 - val_loss: 1.5918


1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x00000241CF5C3600> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Accuracy: 0.5789473684210527

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       1.00      0.71      0.83         7
           2       0.83      0.83      0.83         6
           3       0.56      0.83      0.67         6
           4       0.38      0.50      0.43         6
           5       0.50      0.57      0.53         7

    accuracy                           0.58        38
   macro avg       0.54      0.58      0.55        38
weighted avg       0.55      0.58      0.56        38


Confusion Matrix:
 [[0 0 0 2 3 1]
 [0 5 0 0 1 1]
 [0 0 5 1 0 0]
 [0 0 1 5 0 0]
 [1 0 0 0 3 2]
 [1 0 0 1 1 4]]


In [13]:
model.save("concepts_to_true_labels_model.keras")
print("\nModel saved as 'concepts_to_true_labels_model.keras'")


Model saved as 'concepts_to_true_labels_model.keras'
